In [1]:
import cv2
import mediapipe as mp
import math
import numpy as np
from time import sleep
# from ffpyplayer.player import MediaPlayer
# from mediapipe.python.solutions import hands

In [2]:
def dist(x1,y1,x2,y2): # 비교를 위한 거리 계산 함수
    return math.sqrt( math.pow(x1-x2,2) + math.pow(y1-y2,2))

In [3]:
compareIndex = [[18,4],[6,8],[10,12],[14,16],[18,20]] # 마디 index 번호
hand_open = [False,False,False,False,False] # False로 초기화
gesture = [
    [False,True,True,False,False,"Yeah"],
    [True,False,False,False,False,"Good"],
]

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mpHands = mp.solutions.hands
my_hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# 첫번째 영상

In [5]:
#control frame rate
cap = cv2.VideoCapture('../dance/attention.mp4') # jupyter 
# cap = cv2.VideoCapture('./sample_dance/videoplayback.mp4') # vs

#cap = cv2.VideoCapture('C:/Users/yoond/바탕 화면/00CAPSTONE/Capstone-Design/DAIN/sample_dance/videoplayback.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

frame_counter = 0
frameTime = int((1/fps)*1000)  #time of each frame (ms단위, 몇ms당 1frame으로 할지 설정)
print(fps)
extract_time_by_per_frame = 1 #몇프레임 당 한번 측정할지 조절 가능


23.997370151216305


In [6]:
# ###start step2 --> 영상 추출
# #extract_file = open('DAIN/extractsamples/result.txt','w')
# cap = cv2.VideoCapture('../sample_dance/videoplayback.mp4') # jupyter 
# # cap = cv2.VideoCapture('./sample_dance/videoplayback.mp4') # vs

# #cap = cv2.VideoCapture('C:/Users/yoond/바탕 화면/00CAPSTONE/Capstone-Design/DAIN/sample_dance/videoplayback.mp4')
# fps = cap.get(cv2.CAP_PROP_FPS)



result = np.array([]) # 추출된 영상의 전체 넘파이 배열
bone_index = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28] # 필요한 관절 번호


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame_counter += 1 #increase frame counter

        #Recolor image to RGB
        if ret:
            if frame_counter % extract_time_by_per_frame == 0:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                #Detection
                results = pose.process(image)

                #Recolor image RGB to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                #Extract landmarks
                try: #success to extract landmarks
                    landmarks = results.pose_landmarks.landmark
                    
                    ret = []
                    
                    # # 11 12 13 14 15 16 23 24 25 26 27 28
                    for i in bone_index: # 필요한 부분의 관절의 정보만
                        temp = np.array([landmarks[i].x,landmarks[i].y,landmarks[i].z], float)
                        ret = np.append(ret,temp,axis = 0)
                        
                        
#                     for i in range(len(landmarks)):
#                         temp = np.array([landmarks[i].x,landmarks[i].y,landmarks[i].z], float)
#                         ret = np.append(ret,temp,axis = 0)
                    
                    #print(ret)
                    result = np.append(result,ret,axis=0)
                    #print("extracting\n")
                except:
                    pass
        
        else: #no next frame (end point of video)
            break
            #Show video on screen
        cv2.imshow('dance_file',frame)
        
        if cv2.waitKey(frameTime) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    #extract_file.close()





In [7]:
print('*****extract success*****')
z = result.shape[0]/(len(bone_index)*3)
result = np.reshape(result,(int(z),len(bone_index),3)) # frame 수,  tracking좌표 수, xyz 
print(result[0])
print(result.shape)

*****extract success*****
[[ 0.59443653  0.31211594 -0.4140037 ]
 [ 0.36862716  0.31133595 -0.4822472 ]
 [ 0.61208123  0.41514966 -0.23890303]
 [ 0.34927958  0.42163536 -0.38719669]
 [ 0.62444079  0.50745541 -0.37093556]
 [ 0.34446967  0.50966877 -0.538957  ]
 [ 0.54751533  0.49366298  0.01651365]
 [ 0.42726916  0.49389952 -0.01702734]
 [ 0.53012538  0.64128828  0.03779041]
 [ 0.44629905  0.63877809  0.02240073]
 [ 0.51319945  0.75820625  0.44198593]
 [ 0.46011201  0.75520265  0.40289661]]
(146, 12, 3)


In [8]:
print(len(result))

146


# body index
body_index = [
    
    [11,13,15], # left arm
    [12,14,16], # right arm
    [23,25,27], # left leg
    [24,26,28], # right leg
    
    [14,12,24],
    [13,11,23],
    [23,24,26],
    [24,23,25],
    
]
l1 = result[0][11]
l2 = result[0][13]
l3 = result[0][15]


ll1 = l1-l2
ll2 = l3-l2

innerAB = np.dot(ll1,ll2)
AB = np.linalg.norm(ll1) * np.linalg.norm(ll2)
angle = np.arccos(innerAB/AB)
print(angle) # radian
print(angle/np.pi*180)

# 두번째 영상

In [9]:
#control frame rate
cap = cv2.VideoCapture('../save_user_video/attention_rest.mp4') #('../../HSLEE/demo/dance/sample.mp4') # jupyter 
# cap = cv2.VideoCapture('./sample_dance/videoplayback.mp4') # vs

#cap = cv2.VideoCapture('C:/Users/yoond/바탕 화면/00CAPSTONE/Capstone-Design/DAIN/sample_dance/videoplayback.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

frame_counter = 0
frameTime = int((1/fps)*1000)  #time of each frame (ms단위, 몇ms당 1frame으로 할지 설정)
print(fps)
extract_time_by_per_frame = 1 #몇프레임 당 한번 측정할지 조절 가능


23.997473950110514


In [10]:
# ###start step2 --> 영상 추출
# #extract_file = open('DAIN/extractsamples/result.txt','w')
# cap = cv2.VideoCapture('../sample_dance/videoplayback.mp4') # jupyter 
# # cap = cv2.VideoCapture('./sample_dance/videoplayback.mp4') # vs

# #cap = cv2.VideoCapture('C:/Users/yoond/바탕 화면/00CAPSTONE/Capstone-Design/DAIN/sample_dance/videoplayback.mp4')
# fps = cap.get(cv2.CAP_PROP_FPS)



user_video_result = np.array([]) # 추출된 영상의 전체 넘파이 배열


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame_counter += 1 #increase frame counter

        #Recolor image to RGB
        if ret:
            if frame_counter % extract_time_by_per_frame == 0:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                #Detection
                results = pose.process(image)

                #Recolor image RGB to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                #Extract landmarks
                try: #success to extract landmarks
                    landmarks = results.pose_landmarks.landmark
                    
                    ret = []
                    
                    # # 11 12 13 14 15 16 23 24 25 26 27 28
                    for i in bone_index: # 필요한 부분의 관절의 정보만
                        temp = np.array([landmarks[i].x,landmarks[i].y,landmarks[i].z], float)
                        ret = np.append(ret,temp,axis = 0)
                        
                        
#                     for i in range(len(landmarks)):
#                         temp = np.array([landmarks[i].x,landmarks[i].y,landmarks[i].z], float)
#                         ret = np.append(ret,temp,axis = 0)
                    
                    #print(ret)
                    user_video_result = np.append(user_video_result,ret,axis=0)
                    #print("extracting\n")
                except:
                    pass
        
        else: #no next frame (end point of video)
            break
            #Show video on screen
        cv2.imshow('dance_file',frame)
        
        if cv2.waitKey(frameTime) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    #extract_file.close()





In [11]:
print('*****extract success*****')
z = user_video_result.shape[0]/(len(bone_index)*3)
user_video_result = np.reshape(user_video_result,(int(z),len(bone_index),3)) # frame 수,  tracking좌표 수, xyz 
print(user_video_result[0])
print(user_video_result.shape)

*****extract success*****
[[ 0.59551901  0.30520397 -0.32198745]
 [ 0.39104757  0.30594528 -0.36191466]
 [ 0.60711533  0.40118402 -0.19735564]
 [ 0.37613732  0.4031454  -0.26750457]
 [ 0.60751474  0.48029205 -0.38648674]
 [ 0.37011957  0.48156863 -0.4467569 ]
 [ 0.53668082  0.47034797  0.01123478]
 [ 0.43525684  0.46816608 -0.0115667 ]
 [ 0.5300858   0.60044068 -0.01990704]
 [ 0.45353487  0.59873593 -0.01237494]
 [ 0.51100743  0.70961845  0.41195133]
 [ 0.47295794  0.71164304  0.38295519]]
(152, 12, 3)


In [12]:
len(user_video_result)

152

In [13]:
body_index = [
    [0,2,4], # left arm
    [1,3,4], # right arm
    [6,8,10], # left leg
    [7,9,11], # right leg

    [3,1,7], # right shoulder
    [2,0,6], # left shoulder
    [6,7,9], # right pelvis
    [7,6,8], # left pelvis
]

In [14]:
# user_video_result = result # 녹화된 비디오


# 프레임 수가 작은 것을 기준으로 
length = len(result)
print(length)

if( len(result) >= len(user_video_result)):
    length = len(user_video_result)
    print(len(user_video_result))

print(length)

146
146


In [47]:
print(a,b)

146 226


In [51]:
# 뒤에서부터 비교하기
a = len(result) # 146
b = len(user_video_result) # 226

# 영상 짧아도 무조건 b가 유저 영상
# 대신 짧은 값을 length로 가지고 있으니 그걸 가지고 하면 될 듯함
# if( a < b ):
#     b,a = a,b

# 타임스탬프 체크용 카운터
time_counter = 0

txt = open('../coordinates/comment.txt', 'w')
    
total_point = 0 # 총 포인트
frame_point = 0 # 프레임에서 체크한 포인트

joint_counter = np.zeros(8) # 틀린 프레임 누적하면서 어느 부분이 많이 틀렸는지 카운트

for n in range(b - 1, b - 1 - length , -1):
#     print("new frame")

    frame_point = 0
    joint_index = 0
        
    for index in body_index:
        l1 = result[n+(a-b)][index[0]]
        l2 = result[n+(a-b)][index[1]]
        l3 = result[n+(a-b)][index[2]]

        ll1 = l1-l2
        ll2 = l3-l2
        
        innerREAL = np.dot(ll1,ll2)
        REAL = np.linalg.norm(ll1) * np.linalg.norm(ll2)
        angleREAL = np.arccos(innerREAL/REAL)/np.pi*180
        #print(angleREAL/np.pi*180)


        u1 = user_video_result[n][index[0]]
        u2 = user_video_result[n][index[1]]
        u3 = user_video_result[n][index[2]]

        uu1 = u1-u2
        uu2 = u3-u2
        
        innerUSER = np.dot(uu1,uu2)
        USER = np.linalg.norm(uu1) * np.linalg.norm(uu2)
        angleUSER = np.arccos(innerUSER/USER)/np.pi*180
        
        # 동영상 각도 / 유저 각도 체크해보는 print
#         print(angleREAL)
#         print(angleUSER)
#         print()

        if angleUSER <= angleREAL + 40.0 and angleUSER >= angleREAL - 40.0:
            total_point+=1
            frame_point+=1
        elif (angleREAL + 40.0 < angleUSER and angleUSER <= angleREAL + 60.0) and (angleUSER >= angleREAL - 60.0 and angleREAL - 40.0 > angleUSER):
            total_point+=0.5
            frame_point+=0.5
            joint_counter[joint_index] += 1
#             print("0.5!")
        else:
            joint_counter[joint_index] += 1
#             print("x")
        
        joint_index += 1
        
        
    if(frame_point == 8):
        # 정확도 측정 요소 8개 모두 맞았을 때
        
        if(time_counter >= 20):
            # 20 프레임 이상 연속해서 틀리다가 다시 맞게 추었을 때
            timestamp_min_frame_num = n
                
            # 두 영상의 fps를 맞춰서 같은 변수로 저장했으므로 그대로 사용해도 된다.(가정)
            
            # 방금까지 측정된 부분의 시간(시작점)
            timestamp_min_time_sec = (int)(timestamp_min_frame_num / fps)
            timestamp_min_time_msec = (float)((timestamp_min_frame_num % fps)/fps)
            timestamp_min_time = (float)(timestamp_min_time_sec) + timestamp_min_time_msec
            timestamp_min_time_floor = math.floor(timestamp_min_time)
                
            # 방금까지 측정된 부분의 시간(종료지점)
            timestamp_max_time_sec = (int)(timestamp_max_frame_num / fps)
            timestamp_max_time_msec = (float)((timestamp_max_frame_num % fps)/fps)
            timestamp_max_time = (float)(timestamp_max_time_sec) + timestamp_max_time_msec
            timestamp_max_time_ceil = math.ceil(timestamp_max_time)
            
            # 관절 좌표 어느부분(2부위)이 많이 틀렸는지 체크해서 알려주기
            joint_counter_sort = np.sort(joint_counter)[::-1]
            jointstamp_temp = ""
            
            already_check = -1
            i = 0
            while True:
                for j in range(0,8):
                    if(joint_counter[j] == joint_counter_sort[i]):
                        if((j == 0 or j == 5) and already_check != 0):
                            jointstamp_temp = "왼쪽 팔"
                            already_check = 0
                            break
                        elif((j == 1 or j == 4) and already_check != 1):
                            jointstamp_temp = "오른쪽 팔"
                            already_check = 1
                            break
                        elif((j == 2 or j == 7) and already_check != 2):
                            jointstamp_temp = "왼쪽 다리"
                            already_check = 2
                            break
                        elif((j == 3 or j == 6) and already_check != 3):
                            jointstamp_temp = "오른쪽 다리"
                            already_check = 3
                            break
                else:
                    break
                    
                            
                if(i == 0):
                    jointstamp = jointstamp_temp
                if(i != 0 and jointstamp != jointstamp_temp):
                    jointstamp = jointstamp + ", "+ jointstamp_temp
                    break
                if(i == 7):
                    break
                    
                i += 1
                
            # 텍스트 파일에 틀린 시간 적어줌
            timestamp = "대략 " + str(timestamp_min_time_floor) + "초 ~ " + str(timestamp_max_time_ceil) + "초 : "
            comment = timestamp + jointstamp + " 부분을 잘 보세요."
            txt.write(comment)
            txt.write("\n")
            
#             print("wrong -> correct min")
#             print(n)
                
        # 초기화
#         print("min")
#         print(n)
        time_counter = 0
        joint_counter[:] = 0
    else:
        # 정확도 측정 요소 8개 중 하나라도 틀렸다면
        
        if(time_counter == 0):
            # 틀린 자세 처음 count
            timestamp_max_frame_num = n
            
#             print("max")
#             print(n)
        time_counter += 1
    
    if(n == b - length and time_counter >= 20):
        # 마지막 프레임이면서 20 프레임 이상 연속해서 틀리다가 다시 맞게 추었을 때
        timestamp_min_frame_num = n
                
        # 두 영상의 fps를 맞춰서 같은 변수로 저장했으므로 그대로 사용해도 된다.(가정)
        
        # 방금까지 측정된 부분의 시간(시작점)
        timestamp_min_time_sec = (int)(timestamp_min_frame_num / fps)
        timestamp_min_time_msec = (float)((timestamp_min_frame_num % fps)/fps)
        timestamp_min_time = (float)(timestamp_min_time_sec) + timestamp_min_time_msec
        timestamp_min_time_floor = math.floor(timestamp_min_time)
                
        # 방금까지 측정된 부분의 시간(종료지점)
        timestamp_max_time_sec = (int)(timestamp_max_frame_num / fps)
        timestamp_max_time_msec = (float)((timestamp_max_frame_num % fps)/fps)
        timestamp_max_time = (float)(timestamp_max_time_sec) + timestamp_max_time_msec
        timestamp_max_time_ceil = math.ceil(timestamp_max_time)
                
        # 관절 좌표 어느부분(2부위)이 많이 틀렸는지 체크해서 알려주기
        joint_counter_sort = np.sort(joint_counter)[::-1]
        jointstamp_temp = ""
            
        already_check = -1
        i = 0
        while True:
            for j in range(0,8):
                if(joint_counter[j] == joint_counter_sort[i]):
                    if((j == 0 or j == 5) and already_check != 0):
                        jointstamp_temp = "왼쪽 팔"
                        already_check = 0
                        break
                    elif((j == 1 or j == 4) and already_check != 1):
                        jointstamp_temp = "오른쪽 팔"
                        already_check = 1
                        break
                    elif((j == 2 or j == 7) and already_check != 2):
                        jointstamp_temp = "왼쪽 다리"
                        already_check = 2
                        break
                    elif((j == 3 or j == 6) and already_check != 3):
                        jointstamp_temp = "오른쪽 다리"
                        already_check = 3
                        break
            else:
                break
                    
                            
            if(i == 0):
                jointstamp = jointstamp_temp
            if(i != 0 and jointstamp != jointstamp_temp):
                jointstamp = jointstamp + ", "+ jointstamp_temp
                break
            if(i == 7):
                break
                    
            i += 1
                
        # 텍스트 파일에 틀린 시간 적어줌
        timestamp = "대략 " + str(timestamp_min_time_floor) + "초 ~ " + str(timestamp_max_time_ceil) + "초 : "
        comment = timestamp + jointstamp + " 부분을 잘 보세요."
        txt.write(comment)
        txt.write("\n")
            
#         print("wrong -> correct min")
#         print(n)
        
txt.close()
        
 # 8 : 비교 각도의 개수 len(result) : frame 개수

In [52]:
print(total_point)
print((total_point / (8 * b ) * 100))

1150
94.57236842105263


for num, r in enumerate(result):
    
    for index in body_index:
        l1 = r[index[0]]
        l2 = r[index[1]]
        l3 = r[index[2]]
        
        ll1 = l1-l2
        ll2 = l3-l2
        
        innerAB = np.dot(ll1,ll2)
        AB = np.linalg.norm(ll1) * np.linalg.norm(ll2)
        angle = np.arccos(innerAB/AB)
        print(angle/np.pi*180)
    
    print('\n')
    
    